In [7]:
import sys
import os

project_dir = os.getcwd()
parent_dir = os.path.dirname(project_dir)
sys.path.insert(0, parent_dir)

In [8]:
from gaitor_function_calling.data.prompting_utils import uberAsd
print(uberAsd)

ASDASD
